In [37]:
# 1-1. 데이터 가져오기 

import pandas as pd 
import matplotlib.pyplot as plt 


socar = pd.read_csv('../../0. raw_data/insurance_fraud_detect_data.csv')
socar 

,fraud_YN,car_model,sharing_type,age_group,has_previous_accident,cumulative_use_count,b2b,accident_ratio,pf_type,socarpass,...,insure_cost,accident_location,car_part1,car_part2,repair_cnt,acc_type1,insurance_site_aid_YN,police_site_aid_YN,total_prsn_cnt,test_set
0,0,2,1,3,0,2,0,100,2,0,...,0,1,0,0,1,0,0,0,-1,0
1,0,1,0,4,0,4,0,0,3,1,...,0,1,1,1,2,1,1,1,0,1
2,0,1,0,4,1,4,0,100,3,1,...,12925040,3,1,0,2,1,1,1,0,1
3,0,3,1,3,0,4,0,0,2,0,...,0,3,1,0,1,0,0,0,-1,0
4,0,1,0,1,0,1,0,100,1,0,...,1209410,1,0,0,2,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,0,2,0,2,0,2,0,100,2,0,...,0,1,1,0,1,0,0,0,-1,0
15996,0,2,0,2,1,4,0,100,1,1,...,0,2,1,0,1,0,0,0,-1,0
15997,0,2,1,2,0,4,0,100,1,0,...,0,0,0,0,1,0,0,0,-1,1
15998,0,2,0,2,0,2,0,100,2,0,...,0,4,1,0,1,0,0,0,-1,0


In [38]:
# 테스트 셋 분리

df = socar[socar['test_set']==0]
df_test = socar[socar['test_set']==1]

In [39]:
# test_set 분리 
X_test = df_test.drop('fraud_YN', axis = 1)
y_test = df_test['fraud_YN']

In [40]:
# train-valid 분리 

from sklearn.model_selection import train_test_split

y_target = df['fraud_YN']
X_features = df.drop('fraud_YN', axis = 1)

X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size = 0.2, random_state = 13, stratify = y_target)

# 비율 확인 
print(y_train.value_counts()/y_train.shape[0]*100) 
print(y_valid.value_counts()/y_valid.shape[0]*100)

0    99.73794
1     0.26206
Name: fraud_YN, dtype: float64
0    99.728261
1     0.271739
Name: fraud_YN, dtype: float64


In [41]:
# 학습 / 예측 / 평가 함수 

# 평가 함수 

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

def get_clf_eval(y_test, pred):
    # confusion = confusion_matrix(y_test, pred) #오차행렬
    accuracy = accuracy_score(y_test,pred) # 정확도 
    precision = precision_score(y_test,pred) # 정밀도
    recall = recall_score(y_test,pred) # 재현율 
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test,pred)
    return accuracy, precision, recall, f1, roc_auc

# 성능 DF 반환 

def get_result(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)

    return get_clf_eval(y_test, pred)

# 다수의 모델 성능 비교 

def get_result_df(models, model_names, X_train, y_train, X_test, y_test):
    col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc']
    tmp = []
    for model in models:
        tmp.append(get_result(model, X_train, y_train, X_test, y_test))

    return pd.DataFrame(tmp, columns=col_names, index=model_names)


# 성능 확인 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import LinearSVC


# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()


# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.997283,0.0,0.0,0.0,0.500000
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.990295,0.0,0.0,0.0,0.496497


## Trial #1
- ssscale / pca(n=2) / smote-teen

In [42]:
# 모델 객체를 predict(X_test) 시켜줄 것 
df_1 = df.copy()

In [43]:
y_target = df_1['fraud_YN']
X_features = df_1.drop('fraud_YN', axis=1)

In [44]:
y_target = df_1['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [45]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-0.900373,-0.056158,0
12875,-1.223349,1.349421,0
12876,-0.676312,0.382668,0
12877,-1.383718,0.656705,0
12878,-1.286982,-1.024179,0


In [46]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.995730,0.0,0.0,0.0,0.499221
SVC,0.997283,0.0,0.0,0.0,0.500000


In [47]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.650621,0.005537,0.714286,0.010989,0.682367
DecisionTree,0.490683,0.001528,0.285714,0.003040,0.388478
RandomFore,0.779891,0.001779,0.142857,0.003515,0.462242
LGBM,0.789596,0.003711,0.285714,0.007326,0.538342
SVC,0.664984,0.005774,0.714286,0.011455,0.689568


In [48]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2']

DF_test_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [49]:
# 다수의 모델 테스트 성능 비교 함수 

def get_test_result(model, X_test_scaled, y_test_scaled):
    pred = model.predict(X_test_scaled)
    return get_clf_eval(y_test_scaled, pred)

def get_test_df(models, model_names, X_test_scaled, y_test_scaled):
    col_names = ['Accuracy', 'precision', 'recall', 'f1', 'roc']
    tmp = []
    for model in models:
        tmp.append(get_test_result(model, X_test_scaled,y_test_scaled))
    return pd.DataFrame(tmp, columns=col_names, index=model_names)


In [50]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.642208,0.004335,0.588235,0.008606,0.615293
DecisionTree,0.491731,0.003810,0.735294,0.007580,0.613190
RandomFore,0.812874,0.011513,0.823529,0.022709,0.818187
LGBM,0.821104,0.012457,0.852941,0.024555,0.836981
SVC,0.656573,0.004516,0.588235,0.008963,0.622494


## Trial #2
## ssscale / pca(n=2) / smote-tomek

In [51]:
df_2 = df.copy()

y_target = df_2['fraud_YN']
X_features = df_2.drop('fraud_YN', axis = 1)

y_target = df_2['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [52]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=2)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()


,component_1,component_2,target
12874,-0.900417,-0.064090,0
12875,-1.223360,1.345519,0
12876,-0.676325,0.379296,0
12877,-1.383702,0.657022,0
12878,-1.286989,-1.024210,0


In [53]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)

results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.996894,0.0,0.0,0.0,0.499805
LGBM,0.996118,0.0,0.0,0.0,0.499416
SVC,0.997283,0.0,0.0,0.0,0.500000


In [54]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTETomek
tmk = SMOTETomek(random_state=13)
X_train_tmk, y_train_tmk = tmk.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_tmk, y_train_tmk, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.684394,0.006127,0.714286,0.012151,0.699299
DecisionTree,0.522904,0.000817,0.142857,0.001625,0.333398
RandomFore,0.828804,0.004566,0.285714,0.008989,0.557999
LGBM,0.839286,0.004866,0.285714,0.009569,0.563254
SVC,0.697205,0.006386,0.714286,0.012658,0.705722


In [55]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [56]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.492470,0.010685,0.629630,0.021014,0.560452
DecisionTree,0.538289,0.006974,0.370370,0.013689,0.455062
RandomFore,0.798782,0.004942,0.111111,0.009464,0.457947
LGBM,0.811278,0.008741,0.185185,0.016694,0.500964
SVC,0.497597,0.010794,0.629630,0.021223,0.563037


## Trial #3
## ssscale / pca(n=3) / smoteenn

In [57]:
df_3 = df.copy()

y_target = df_3['fraud_YN']
X_features = df_3.drop('fraud_YN', axis=1)

y_target = df_3['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [58]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,target
12874,-0.900765,-0.069389,1.019894,0
12875,-1.223324,1.353651,0.047742,0
12876,-0.676864,0.363593,1.495340,0
12877,-1.383857,0.646814,-0.843939,0
12878,-1.286942,-1.023315,-1.606073,0


In [59]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996506,0.0,0.0,0.0,0.499611
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.997283,0.0,0.0,0.0,0.500000
SVC,0.997283,0.0,0.0,0.0,0.500000


In [60]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTEENN
sme = SMOTEENN(random_state=13)
X_train_sme, y_train_sme = sme.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_sme, y_train_sme, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.623059,0.005133,0.714286,0.010194,0.668548
DecisionTree,0.442935,0.002786,0.571429,0.005544,0.507007
RandomFore,0.914596,0.009217,0.285714,0.017857,0.601012
LGBM,0.907220,0.012605,0.428571,0.024490,0.668548
SVC,0.630435,0.005236,0.714286,0.010395,0.672246


In [61]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_test_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [62]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.622486,0.004922,0.705882,0.009776,0.664074
DecisionTree,0.460828,0.004162,0.852941,0.008283,0.656365
RandomFore,0.930196,0.031419,0.852941,0.060606,0.891671
LGBM,0.926780,0.030960,0.882353,0.059821,0.904625
SVC,0.630018,0.005022,0.705882,0.009973,0.667850


## Trial #4
## ssscale / pca(n=3) / smote-tomek

In [63]:
df_4 = df.copy()

y_target = df_4['fraud_YN']
X_features = df_4.drop('fraud_YN', axis=1)

y_target = df_1['fraud_YN'].reset_index()
y_target = y_target.drop('index', axis=1)

In [64]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(X_features)

pca = PCA(n_components=3)
pca.fit(df_scaled)
df_pca = pca.transform(df_scaled)

pca_columns = ['component_1','component_2','component_3']

DF_pca = pd.DataFrame(df_pca, columns=pca_columns)
DF_pca['target'] = y_target 
DF_pca.tail()

,component_1,component_2,component_3,target
12874,-0.900645,-0.052143,1.062274,0
12875,-1.223540,1.370077,0.081288,0
12876,-0.676121,0.357878,1.487625,0
12877,-1.383486,0.650616,-0.855555,0
12878,-1.286765,-1.045111,-1.628164,0


In [65]:
y_target = DF_pca['target']
X_features = DF_pca.drop('target', axis= 1)


X_train, X_valid, y_train, y_valid = train_test_split(X_features, y_target, test_size =0.2, random_state=13, stratify= y_target)

# 모델 객체 생성 
lr_clf = LogisticRegression(random_state=13)
dt_clf = DecisionTreeClassifier(random_state=13, max_depth=4)
rf_clf = RandomForestClassifier(random_state=13, n_jobs = -1, n_estimators= 100 )
lgbm_clf = LGBMClassifier(n_estimators=1000, num_leaves=64, n_jobs=-1, boost_from_average = False)
svm_clf = LinearSVC()

# 학습 진행 
models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train, y_train, X_valid, y_valid)
results 


,Accuracy,precision,recall,f1,roc
LogiReg,0.997283,0.0,0.0,0.0,0.500000
DecisionTree,0.996894,0.0,0.0,0.0,0.499805
RandomFore,0.997283,0.0,0.0,0.0,0.500000
LGBM,0.996894,0.0,0.0,0.0,0.499805
SVC,0.997283,0.0,0.0,0.0,0.500000


In [66]:
# 복합 샘플링 적용 

from imblearn.combine import SMOTETomek
tmk = SMOTETomek(random_state=13)
X_train_tmk, y_train_tmk = tmk.fit_sample(X_train, y_train)


model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_result_df(models, model_names, X_train_tmk, y_train_tmk, X_valid, y_valid)

results

,Accuracy,precision,recall,f1,roc
LogiReg,0.677019,0.005988,0.714286,0.011876,0.695601
DecisionTree,0.481366,0.002992,0.571429,0.005952,0.526275
RandomFore,0.937112,0.006369,0.142857,0.012195,0.541067
LGBM,0.940994,0.006803,0.142857,0.012987,0.543013
SVC,0.683618,0.006112,0.714286,0.012121,0.698910


In [67]:
# 테스트셋 적용 
# PCA / ss 

df_test_scaled = scaler.transform(X_test)
df_test_pca = pca.transform(df_test_scaled)

pca_columns = ['component_1','component_2', 'component_3']

DF_test_pca = pd.DataFrame(df_test_pca, columns=pca_columns)
DF_test_pca['target'] = y_target 
DF_test_pca.tail()

X_test_scaled = DF_test_pca.drop('target', axis=1)
y_test_scaled = DF_test_pca['target']

In [70]:
# 테스트 셋 모델 적용 

models=[lr_clf,dt_clf,rf_clf,lgbm_clf,svm_clf] 
model_names = ['LogiReg', 'DecisionTree', 'RandomFore','LGBM','SVC']
results = get_test_df(models,model_names, X_test_scaled, y_test_scaled)
results 

,Accuracy,precision,recall,f1,roc
LogiReg,0.553989,0.009346,0.481481,0.018336,0.518052
DecisionTree,0.506248,0.005875,0.333333,0.011546,0.420545
RandomFore,0.935598,0.005682,0.037037,0.009852,0.490238
LGBM,0.940083,0.000000,0.000000,0.000000,0.474144
SVC,0.557834,0.010138,0.518519,0.019886,0.538348


In [68]:
# ssscale / pca(n=3) / smote-tomek / repair_cost 전처리 

In [69]:
# robust / pca(n=3) / smote-tomek